In [ ]:
# This code can only run on colab
# model training
from google.colab import drive
drive.mount('/content/drive')
!pip install simplejson
%run /content/drive/MyDrive/HumanActivityRecognition-master/main.py

In [ ]:
# model test
%run /content/drive/MyDrive/HumanActivityRecognition-master/test_model.py

In [12]:
from models.svcca import svd_reduction
from models.cca import get_cca_similarity#, cca_distance
import torch

def get_similarity(epoch1, epoch2, l1, l2,dataset1, dataset2):
    '''
    This code can run after training the model and specifying the args.svcca_epochs.
    Compare the representation of layer l1 from dataset1 and layer l2 from dataset2 at epoch1
    or compare the representation of layer l1 from dataset1 at epoch1 and at epoch 2
    Return the svcca similarity
    '''
    layer1 = torch.load('/content/drive/MyDrive/' + dataset1 + '/Epoch'+ str(epoch1) + l1 +'.pt')
    layer2 = torch.load('/content/drive/MyDrive/' + dataset2 + '/Epoch'+ str(epoch2) + l2 +'.pt')
    layer1 = svd_reduction(layer1)
    layer2 = svd_reduction(layer2)  
    acts1 = layer1.cpu().detach().numpy()
    acts2 = layer2.cpu().detach().numpy()

    # acts1 = acts1 - np.mean(acts1, axis=1)
    
    if l1 == 'fc' and l2 == 'fc':
        f_results = get_cca_similarity(acts1.T, acts2.T, epsilon=1e-10, verbose=False, compute_dirns=True)
        return sum(f_results['cca_coef1'])/len(f_results['cca_coef1'])
    elif l2 == 'fc':
        acts1 = np.mean(acts1, axis=2)
        return cca_distance(acts1, acts2)
    elif l1 == 'fc':
        acts2 = np.mean(acts2, axis=2)
        return cca_distance(acts1, acts2)  
    elif acts1.shape[-1] != acts2.shape[-1]:
        num_datapoints, h, channels = acts1.shape
        f_acts1 = acts1.reshape((channels*h, num_datapoints))

        num_datapoints, h, channels = acts2.shape
        f_acts2 = acts2.reshape((channels*h, num_datapoints))

        return cca_distance(f_acts1.T, f_acts2.T)
    else:
        num_datapoints, channels, h = acts1.shape
        f_acts1 = acts1.reshape((num_datapoints*h, channels))

        num_datapoints, channels,h = acts2.shape
        f_acts2 = acts2.reshape((num_datapoints*h, channels))

        f_results = get_cca_similarity(f_acts1.T, f_acts2.T, epsilon=1e-10, verbose=False, compute_dirns=True)
        return sum(f_results['cca_coef1'])/len(f_results['cca_coef1'])



# get_similarity(150,150, 'layer1_1_conv1', 'layer1_1_conv1', 'Rep_daph', 'Rep_pamap2')

## Heatmap plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def heatMapCorr(layer_names = [], idxs = [], dataset = None):
    corr_arr = []
    for layer_name1 in layer_names:
        corr = []
        for layer_name2 in layer_names:
            r = get_similarity(idxs[0], idxs[1], layer_name1, layer_name2,dataset)
            print(layer_name1 + layer_name2 + " The SVCCA score is " + str(r))
            corr.append(r)
        corr_arr.append(corr)

    return corr_arr

layer_names = ['conv1', 'layer1_0_conv1', 'layer1_0_conv2',
                        'layer1_1_conv1', 'layer1_1_conv2',
                        'layer2_0_conv1', 'layer2_0_conv2',
                        'layer2_1_conv1', 'layer2_1_conv2',
                        'layer3_0_conv1', 'layer3_0_conv2',
                        'layer3_1_conv1', 'layer3_1_conv2',
                        'layer4_0_conv1', 'layer4_0_conv2',
                        'layer4_1_conv1', 'layer4_1_conv2',
                        'fc']

idxs = [[10,11],[150,151],[397,398]]

corr_arr = heatMapCorr(layer_names, idxs[1], dataset='Rep_opp')

In [ ]:

corr = corr_arr
df = pd.DataFrame(corr_arr)
df.columns = layer_names
df.index = layer_names

f, ax1 = plt.subplots(figsize = (10, 8))
# cubehelix map 
cmap = sns.cubehelix_palette(start = 1.5, rot = 3, gamma=0.8, as_cmap = True)
sns.heatmap(df, linewidths = 0.05, ax = ax1, vmax=1, vmin=0, cmap='YlOrRd')
ax1.set_title('100% trained')
ax1.set_xlabel('layers')
ax1.set_xticklabels([]) 
ax1.set_ylabel('layers')
f.savefig('100%trained.jpg', bbox_inches='tight')